In [1]:
import xarray as xr
import pandas as pd
from datetime import datetime
from datetime import timedelta
import numpy as np
import matplotlib.pyplot as plt
from metpy.units import units
from metpy import calc as mpcalc

In [2]:
# Read in the data 
file = '/Users/georgiaford/sggj_data.csv'
instrumentation_project_dataset = pd.read_csv(file)
gill_dataset = instrumentation_project_dataset
# time is in seconds since 1904
# code that fixes time

file = 'sggj_data.csv' 
gill_dataset = pd.read_csv(file)
dataset_time_array = np.array(gill_dataset['datetime'])
dataset_time_array_numpy = dataset_time_array.astype('datetime64[s]')
time_correction = datetime(1970, 1, 1, 0, 0, 0) - datetime(1904, 1, 1, 0, 0, 0)
time_correction_numpy = np.array([time_correction]).astype('timedelta64[s]')[0]
datetime_proper = dataset_time_array_numpy - time_correction_numpy
gill_dataset['corrected_dts'] = datetime_proper
gill_dataset['youngU'], gill_dataset['youngV'] = mpcalc.wind_components(gill_dataset['youngWS'].values * (units.meters/units.seconds),
                                                              gill_dataset['youngWD'].values * units.deg)
gill_dataset['gillU'], gill_dataset['gillV'] = mpcalc.wind_components(gill_dataset['gillWS'].values * (units.meters/units.seconds),
                                                            gill_dataset['gillWD'].values * units.deg)
gill_dataset['ftU'], gill_dataset['ftV'] = mpcalc.wind_components(gill_dataset['ftWS'].values * (units.meters/units.seconds),
                                                          gill_dataset['ftWD'].values * units.deg)
gill_dataset.set_index('corrected_dts', inplace=True, drop=True)
gill_dataset.drop(columns=['datetime'], inplace=True)
gill_dataset

,youngWS,youngWD,gillWS,gillWD,ftWS,ftWD,youngU,youngV,gillU,gillV,ftU,ftV
corrected_dts,,,,,,,,,,,,
2024-04-18 14:28:10,1.802,233.366,1.56,50.0,1.9,65.0,1.446039,1.075256,-1.195029,-1.002749,-1.721985,-0.802975
2024-04-18 14:28:11,2.176,215.749,2.96,78.0,2.4,63.0,1.271296,1.766007,-2.895317,-0.615419,-2.138416,-1.089577
2024-04-18 14:28:12,2.899,232.680,3.92,65.0,3.2,62.0,2.305464,1.757565,-3.552727,-1.656664,-2.825432,-1.502309
2024-04-18 14:28:13,2.833,200.615,3.23,51.0,3.2,49.0,0.997462,2.651596,-2.510181,-2.032705,-2.415071,-2.099389
2024-04-18 14:28:14,2.206,169.306,1.92,64.0,2.9,41.0,-0.409354,2.167687,-1.725685,-0.841673,-1.902571,-2.188658
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-19 14:34:23,5.221,200.142,4.56,58.0,4.3,65.0,1.797841,4.901695,-3.867099,-2.416432,-3.897123,-1.817259
2024-04-19 14:34:24,5.142,192.669,4.87,61.0,4.2,57.0,1.127735,5.016810,-4.259398,-2.361023,-3.522416,-2.287484
2024-04-19 14:34:25,4.912,227.797,3.98,54.0,4.3,56.0,3.638659,3.299682,-3.219888,-2.339385,-3.564862,-2.404529


In [ ]:
# calculate mean 

mean_gill = np.mean(gill_dataset['gillWS'])
gill_ws = gill_dataset['gillWS']

In [ ]:
# GMX560 variance
gill_ws = gill_dataset['gillWS']
gill_ws_prime = gill_ws - np.nanmean(gill_ws)
gill_ws_squared = gill_ws_prime * gill_ws_prime

# to get the variance, we need the mean of gill_ws_squared
gill_ws_series = pd.Series(gill_ws_squared)

# 3 point moving varience
window_size3 = 3
windows3 = gill_ws_series.rolling(window_size3)
moving_averages3 = windows3.mean()
moving_var_list3 = moving_averages3.tolist()
moving_var3 = moving_var_list3[window_size3-1:]
#results under moving variance 3

In [ ]:
# 10 pt moving varience
window_size10 = 10
windows10 = gill_ws_series.rolling(window_size10)
moving_averages10 = windows10.mean()
moving_var_list10 = moving_averages10.tolist()
moving_var10 = moving_var_list10[window_size10-1:]

In [ ]:
plt.figure(figsize = (12, 8))
plt.plot(gill_dataset.index, gill_ws_squared, label='Raw')
plt.plot(gill_dataset.index, moving_var_list3, label='3-pt')
plt.plot(gill_dataset.index, moving_var_list10, label='10-pt')
plt.xlabel('time')
plt.ylabel('wind speed variance(m/s)')
plt.legend()
plt.title('WNS variance time history GMX560')


In [ ]:
#  Gill time history raw data
plt.figure(figsize = (12, 8))
plt.plot(gill_dataset.index, gill_dataset['gillWS'])
plt.xlabel('time')
plt.ylabel('wind speed (m/s)')
plt.title('WNS Time History GMX560')


In [ ]:
# WND u component
plt.figure(figsize = (12, 8))
plt.plot(gill_dataset.index, gill_dataset['gillU'])
plt.xlabel('time')
plt.ylabel('u wind (m/s)')
plt.title('u wind time history GMX560')

In [ ]:
# WND v component
plt.figure(figsize = (12, 8))
plt.plot(gill_dataset.index, gill_dataset['gillV'])
plt.xlabel('time')
plt.ylabel('v wind (m/s)')
plt.title('v wind time history GMX560')